In [1]:
# use packages
using MKL
using Plots
using Gridap, GridapGmsh, Gridap.Geometry, Gridap.Fields

#Pardiso solver is much faster
# using GridapPardiso
# using SparseMatricesCSR

# For image generation.
#using Images, ImageView,ImageFiltering
using ImageFiltering
# using BenchmarkTools

In [2]:
# constant
# Material property
function lame_parameters(E,ν)
    λ = (E*ν)/((1+ν)*(1-2*ν))
    μ = E/(2*(1+ν))
    (λ, μ)
end
  
#Silicon 
const E_Si = 160.0e9
const ν_Si = 0.27
const (λ_Si, μ_Si) = lame_parameters(E_Si, ν_Si) 
const rho_Si = 2329.0     # kg/m^3
σ_Si(ε) = λ_Si*tr(ε)*one(ε) + 2*μ_Si*ε

# #Vacuum 
# const E_va = 0.0
# const ν_va = 0.0
# const (λ_va, μ_va) = lame_parameters(E_va, ν_va) 
# const rho_va = 0.0    # kg/m^3
# σ_va(ε) = λ_va*tr(ε)*one(ε) + 2*μ_va*ε
  
#PDMS
const E_PDMS = 1.0e6
const ν_PDMS = 0.45
const (λ_PDMS, μ_PDMS) = lame_parameters(E_PDMS, ν_PDMS)
const rho_PDMS = 790.0  #kg/m3
σ_PDMS(ε) = λ_PDMS*tr(ε)*one(ε) + 2*μ_PDMS*ε

# frequency
freq = 1000e6
omega = 2*π*  (freq)
δ=1e-6
L0 = 22 * δ # Length of device
H0 = 4 * δ # Width of device
T0 = 1 * δ # Height of device
d_pmlx = 1 * δ  # Thickness of the PML
d_pmly = 0.5 * δ  # Thickness of the PML
d_pmlz = 0.25 * δ  # Thickness of the PML


2.5e-7

In [3]:
# user-defined functions
import Base:real
function real(a :: VectorValue{D, T}) where {D, T<:Number}
    return VectorValue([real(a[i]) for i in 1:D])
end

import Base:imag
function imag(a :: VectorValue{D, T}) where {D<:Int, T<:Number}
    return VectorValue([imag(a[i]) for i in 1:D])
end

import Base:abs
function abs(a :: VectorValue{D, T}) where {D<:Int, T<:Number}
    return VectorValue([abs(a[i]) for i in 1:D])
end


abs (generic function with 21 methods)

In [4]:
# load meshes and view the models
model = GmshDiscreteModel("E:/Work/Postdoc/Quantumphononics/Test/cantilever_3P_20230701.msh")
writevtk(model,"cant_model") # write the model to vtk


Info    : Reading 'E:/Work/Postdoc/Quantumphononics/Test/cantilever_3P_20230701.msh'...
Info    : 99 entities
Info    : 19635 nodes
Info    : 96390 elements
Info    : Done reading 'E:/Work/Postdoc/Quantumphononics/Test/cantilever_3P_20230701.msh'


4-element Vector{Vector{String}}:
 ["cant_model_0.vtu"]
 ["cant_model_1.vtu"]
 ["cant_model_2.vtu"]
 ["cant_model_3.vtu"]

In [5]:
# Define ref/test space
order = 2 # 2nd order Lagrange interpolation

reffe = ReferenceFE(lagrangian,VectorValue{3,ComplexF64},order) # Finite element basis: 2nd order lagrange interpolation of the funciton space

#Linbo: changed to ComplexF64  for have complex number in displacement representing the phase

# V0 = TestFESpace(model,reffe;
#   conformity=:H1,
#   dirichlet_tags=["clamp",],
#   dirichlet_masks=[(true,true,true),])
V = TestFESpace(model,reffe;
  conformity=:H1,)
# g1(x) = VectorValue(0, 0.0, 0.0001) # Dirichlet boundary condition: the third component of the displacement vector is 0.0001
# g1(x) = VectorValue(0, 0.0, 0.0) # Dirichlet boundary condition: the third component of the displacement vector is 0.0001
# U = TrialFESpace(V0,[g1]) # solution function space: Dirichlet: u=g1

U = V # solution function space: Dirichlet: u=g1

UnconstrainedFESpace()

In [6]:
# Define integral region
degree = 2*order # Q: 2 * order of interpolation 
Ω = Triangulation(model) # integral space: bulk; mesh: triangular
dΩ = Measure(Ω,degree) # integral element: gauss-like quadrature in each cell

Ω_d = Triangulation(model, tags="domain1")
dΩ_d = Measure(Ω_d, degree)

Ω_c = Triangulation(model, tags="PortA")
dΩ_c = Measure(Ω_c, degree)
#neumanntags = ["clamp", "PortA", "PortB"]
#Γ = BoundaryTriangulation(model,tags=neumanntags)
#Γ = BoundaryTriangulation(model)
#dΓ = Measure(Γ,degree)

GenericMeasure()

In [7]:
p_reffe = ReferenceFE(lagrangian, Float64, 0)
Q = TestFESpace(Ω_d, p_reffe, vector_type = Vector{Float64})
P = Q
np = num_free_dofs(P) # Number of cells in design region (number of design parameters)
pf_reffe = ReferenceFE(lagrangian, Float64, 1)
Qf = TestFESpace(Ω_d, pf_reffe, vector_type = Vector{Float64})
Pf = Qf
fem_params = (; V, U, Q, P, Qf, Pf, np, Ω, Ω_d, Ω_c, dΩ, dΩ_d, dΩ_c)

(V = UnconstrainedFESpace(), U = UnconstrainedFESpace(), Q = UnconstrainedFESpace(), P = UnconstrainedFESpace(), Qf = UnconstrainedFESpace(), Pf = UnconstrainedFESpace(), np = 96030, Ω = BodyFittedTriangulation(), Ω_d = BodyFittedTriangulation(), Ω_c = BodyFittedTriangulation(), dΩ = GenericMeasure(), dΩ_d = GenericMeasure(), dΩ_c = GenericMeasure())

In [8]:
# PML in 3D
# Parameters
α = 20 # absorption coefficient

L1 = (d_pmlx ,d_pmly ,d_pmlz) # Corrected size
L2 = (L0-d_pmlx ,H0-d_pmly ,T0-d_pmlz) # Corrected size
d_pml = (d_pmlx ,d_pmly ,d_pmlz)

function s_PML(x, α, L1, L2, d_pml)
    u = abs.(Tuple(x))  # get the depth into PML
    return @. ifelse(
        u < L1, 
        1 + (1im*α)*((L1 - u)/d_pml)^2,
        ifelse(
            u > L2, 
            1 + (1im*α)*((u - L2)/d_pml)^2, 
            1.0+0im
        )
    )
end


struct Λ<:Function
    α::Float64
    L1::NTuple{3,Float64}
    L2::NTuple{3,Float64}  # Here we increased the dimension to 3
    d_pml::NTuple{3,Float64}
end

function (Λf::Λ)(x)
    s_x,s_y,s_z = s_PML(x,Λf.α,Λf.L1,Λf.L2,Λf.d_pml)  # Here we added s_z for the z-axis
    return Gridap.TensorValues.SymTensorValue{3, ComplexF64, 6}(
        1/s_x, 0.0, 0.0, 
        1/s_y, 0.0, 
        1/s_z
    )  # Here we added 1/s_z for the z-axis
end
# function (Λf::Λ)(x)
#     s_x,s_y,s_z = s_PML(x,Λf.α,Λf.L1,Λf.L2,Λf.d_pml)  # Here we added s_z for the z-axis
#     return VectorValue(1/s_x,1/s_y,1/s_z)
# end
# Fields.∇(Λf::Λ) = x->TensorValue{3,3,ComplexF64}(1/s_x,0,0, 0,1/s_y,0, 0,0,1/s_z) 
Λf = Λ(α,L1,L2,d_pml)
# Λf = Λ(α,d_pmlx,d_pmly,d_pmlz,L0,H0,T0)

(::Λ) (generic function with 1 method)

In [9]:
# Threshold
# r = 5/sqrt(3)*100               # Filter radius
# β = 32.0                    # β∈[1,∞], threshold sharpness
# η = 0.5                     # η∈[0,1], threshold center

# a_f(r, u, v) = r^2 * (ε(v) ⊙ ε(u))

# function Filter(p0; r, fem_params)
#     ph = FEFunction(fem_params.P, p0)
#     op = AffineFEOperator(fem_params.Pf, fem_params.Qf) do u, v
#         ∫(a_f(r, u, v))fem_params.dΩ_d + ∫(v ⊙ u)fem_params.dΩ_d, ∫(v ⊙ ph)fem_params.dΩ_d
#       end
#     pfh = solve(op)
#     return get_free_dof_values(pfh)
# end

# function Threshold(p0; β, η)
#     return ((tanh(β * η) + tanh(β * (p0 - η))) / (tanh(β * η) + tanh(β * (1.0 - η))))
# end
# a(u,v) = ∫( -(Λf.⋅ε(v)) ⊙ (σ_Si∘(Λf.⋅ε(u))) + rho_Si*omega*omega*u⊙v )*dΩ


Threshold (generic function with 1 method)

In [14]:
using LinearAlgebra
ξd(p, σ_Si, σ_PDMS)= p ⋅ σ_Si + (1-p) ⋅ σ_PDMS
a_base(u,v) = ( -(Λf.⋅ε(v)) ⊙ (σ_PDMS∘(Λf.⋅ε(u))) + rho_PDMS*omega*omega*u⊙v )
# a_design(u, v, pth) = ((p -> ξd(p, σ_Si, σ_va)) ∘ pth) * (ε(v) ⊙ ε(u))
a_design(u, v, pth) = pth * (ε(v) ⊙ ε(u))
function MatrixA(pth; fem_params)
    A_mat = assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫(a_base(u, v))fem_params.dΩ + ∫(a_design(u, v, pth))fem_params.dΩ_d
    end
    return lu(A_mat)
end

MatrixA (generic function with 1 method)

In [15]:
p0 = zeros(Float64,fem_params.np)  # Here we make p=0 everywhere just for illustration purpose
# pf_vec = Filter(p0;r, fem_params)
pfh = FEFunction(fem_params.Pf, p0)
# pth = (pf -> Threshold(pf; β, η)) ∘ pfh
A_mat = MatrixA(pfh; fem_params)
x₀=1.1δ #this should the x coordinate of the center of the cantilever.
f = x -> VectorValue(0.0,0.0,1/(2π) * exp(-(x[1]-x₀)^2/2/δ^2))
b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
u_vec = A_mat \ b_vec
uh = FEFunction(fem_params.U, u_vec)

SingleFieldFEFunction():
 num_cells: 96030
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 6984631305094146892

In [16]:
function MatrixOf(fem_params)
    x0 = VectorValue(20.9δ,-2.75δ,0.5δ)  # Position of the field to be optimized
    return assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫((x->(1/(2*π)*exp(-norm(x - x0)^2 / 2 / δ^2))) * (ε(u) ⊙ ε(v)) )fem_params.dΩ_c
    end
end

MatrixOf (generic function with 1 method)

In [17]:
using ChainRulesCore, Zygote
import ChainRulesCore: rrule
NO_FIELDS = ZeroTangent()

ZeroTangent()

In [ ]:
Dptdpf(pf, β, η) = β * (1.0 - tanh(β * (pf - η))^2) / (tanh(β * η) + tanh(β * (1.0 - η)))

Dξdpf(pf, σ_va, σ_Si, β, η)= (σ_Si-σ_va) * Dptdpf(pf, β, η)

DAdpf(u, v, pfh;  σ_va, σ_Si, β, η) = ((p -> Dξdpf(p, σ_va, σ_Si, β, η)) ∘ pfh) * (ε(v) ⊙ ε(u))

In [ ]:
function gf_pf(pf_vec; β, η, f,fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; fem_params)
    b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
    u_vec = A_mat \ b_vec

    O_mat = MatrixOf(fem_params)
    real(u_vec' * O_mat * u_vec)
end

function rrule(::typeof(gf_pf), pf_vec; σ_va, σ_Si, β, η,  f,fem_params)
    function U_pullback(dgdg)
      NO_FIELDS, dgdg * Dgfdpf(pf_vec; σ_va, σ_Si, β, η, f,fem_params)
    end
    gf_pf(pf_vec; β, η, f,fem_params), U_pullback
end

function Dgfdpf(pf_vec; σ_va, σ_Si, β, η, f,fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; fem_params)
    b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
    u_vec = A_mat \ b_vec
    O_mat = MatrixOf(fem_params)

    uh = FEFunction(fem_params.U, u_vec)
    w_vec =  A_mat' \ (O_mat * u_vec)
    wconjh = FEFunction(fem_params.U, conj(w_vec))

    l_temp(dp) = ∫(real(-2 * DAdpf(uh, wconjh, pfh; σ_va, σ_Si,  β, η)) * dp)fem_params.dΩ_d
    dgfdpf = assemble_vector(l_temp, fem_params.Pf)
    return dgfdpf
end



In [ ]:
function pf_p0(p0; r, fem_params)
  pf_vec = Filter(p0; r, fem_params)
  pf_vec
end

function rrule(::typeof(pf_p0), p0; r, fem_params)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf; r, fem_params)
  end
  pf_p0(p0; r, fem_params), pf_pullback
end

function Dgdp(dgdpf; r, fem_params)
  Af = assemble_matrix(fem_params.Pf, fem_params.Qf) do u, v
      ∫(a_f(r, u, v))fem_params.dΩ_d + ∫(v * u)fem_params.dΩ_d
  end
  wvec = Af' \ dgdpf
  wh = FEFunction(fem_params.Pf, wvec)
  l_temp(dp) = ∫(wh * dp)fem_params.dΩ_d
  return assemble_vector(l_temp, fem_params.P)
end

In [ ]:
function gf_p(p0::Vector; r, β, η,f,fem_params)
    pf_vec = pf_p0(p0; r, fem_params)
    gf_pf(pf_vec; β, η, f, fem_params)
end

function gf_p(p0::Vector, grad::Vector; r, β, η, f,fem_params)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> gf_p(p; r, β, η, f,fem_params), p0)
        grad[:] = dgdp
    end
    gvalue = gf_p(p0::Vector; r, β, η, f,fem_params)
    open("gvalue.txt", "a") do io
        write(io, "$gvalue \n")
    end
    gvalue
end

In [ ]:
p0 = rand(fem_params.np)
δp = rand(fem_params.np)*1e-6
grad = zeros(fem_params.np)

g0 = gf_p(p0, grad; r, β, η, f,fem_params)
g1 = gf_p(p0+δp, []; r, β, η, f,fem_params)
g1-g0, grad'*δp

In [ ]:
using NLopt

function gf_p_optimize(p_init; r, β, η, TOL = 1e-4, MAX_ITER = 500, f, fem_params)
    ##################### Optimize #################
    opt = Opt(:LD_MMA, fem_params.np)
    opt.lower_bounds = 0
    opt.upper_bounds = 1
    opt.ftol_rel = TOL
    opt.maxeval = MAX_ITER
    opt.max_objective = (p0, grad) -> gf_p(p0, grad; r, β, η, f, fem_params)

    (g_opt, p_opt, ret) = optimize(opt, p_init)
    @show numevals = opt.numevals # the number of function evaluations
    return g_opt, p_opt
end

p_opt = fill(0.4, fem_params.np)   # Initial guess
β_list = [8.0, 16.0, 32.0]

g_opt = 0
TOL = 1e-8
MAX_ITER = 100
for bi = 1 : 3
    β = β_list[bi]
    g_opt, p_temp_opt = gf_p_optimize(p_opt; r, β, η, TOL, MAX_ITER, f, fem_params)
    global p_opt = p_temp_opt
end
@show g_opt

In [ ]:
using NLopt

function gf_p_optimize(p_init; r, β, η, TOL = 1e-4, MAX_ITER = 500, f, fem_params)
    ##################### Optimize #################
    opt = Opt(:LD_MMA, fem_params.np)
    opt.lower_bounds = 0
    opt.upper_bounds = 1
    opt.ftol_rel = TOL
    opt.maxeval = MAX_ITER
    opt.max_objective = (p0, grad) -> gf_p(p0, grad; r, β, η, f, fem_params)

    (g_opt, p_opt, ret) = optimize(opt, p_init)
    @show numevals = opt.numevals # the number of function evaluations
    return g_opt, p_opt
end

p_opt = fill(0.4, fem_params.np)   # Initial guess
β_list = [8.0, 16.0, 32.0]

g_opt = 0
TOL = 1e-6
MAX_ITER = 100
for bi = 1 : 3
    β = β_list[bi]
    g_opt, p_temp_opt = gf_p_optimize(p_opt; r, β, η, TOL, MAX_ITER, f, fem_params)
    global p_opt = p_temp_opt
end
@show g_opt

In [ ]:
using CairoMakie, GridapMakie
p0 = p_opt

pf_vec = pf_p0(p0; r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Threshold(pf; β, η)) ∘ pfh
A_mat = MatrixA(pth; phys_params, fem_params)
f = x -> VectorValue(0.0,0.0,1/(2π) * exp(-(x[1]-x₀)^2/2/δ^2))
b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
u_vec = A_mat \ b_vec
uh = FEFunction(fem_params.U, u_vec)

fig, ax, plt = plot(fem_params.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(1)
ax.title = "Design Shape"
rplot = 110 # Region for plot
limits!(ax, 0, 22δ, 0, -4δ)
save("shape.png", fig)

In [ ]:

x₀=1.1δ #this should the x coordinate of the center of the cantilever.

f = x -> VectorValue(0.0,0.0,1/(2π) * exp(-(x[1]-x₀)^2/2/δ^2))


# l(v) = 0
l(v) = ∫( v⋅f )*dΩ
# a(u,v) = ∫( -ε(v) ⊙ (σ_Si∘(ε(u))) + rho_Si*omega*omega*u⊙v )*dΩ

a(u,v) = ∫( -(Λf.⋅ε(v)) ⊙ (σ_Si∘(Λf.⋅ε(u))) + rho_Si*omega*omega*u⊙v )*dΩ
println("assemble 2")
op = AffineFEOperator(a,l,U,V0)

println("solving 2")
#solve
solver = LinearFESolver()
uh = solve(solver, op)

println("writing solution 1")
writevtk(Ω,"20230922_pureSi_shift_no_PML",cellfields=["uhR"=>real(uh), "rho"=>rho_Si])

println("complete.")

In [ ]:
g = zeros(Float64, 11)

for c in 1:1:11
    g[c] = real(uh(VectorValue(10δ,-2δ,(c-1)*δ/10)))[3]
end

p = plot((0:1:10)*δ/10, g, label="With PML", xlims=(0,10*δ/10), ylims=(-4e-24,4e-24),
         xlabel="z/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:1e-6:5e-6,yticks=-4e-24:2e-24:4e-24)

# Add vertical lines at x = 2e-6 and 18e-6
vline!(p, [1e-6, 3e-6], color=:red, linestyle=:dash,label="")

# Display the updated plot
display(p)

In [ ]:
g = zeros(Float64, 41)

for c in 1:1:41
    g[c] = real(uh(VectorValue(10δ,-(c-1)*δ/10,0.5δ)))[3]
end

# p = plot((0:1:40)*δ/10, g, label="With PML", xlims=(0,40*δ/10), ylims=(-4e-24,4e-24),
#          xlabel="y/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:1e-6:5e-6,yticks=-4e-24:2e-24:4e-24)
p = plot((0:1:40)*δ/10, g, label="With PML", xlims=(0,40*δ/10),ylims=(0,2e-24),
         xlabel="y/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:1e-6:4e-6)
# Add vertical lines at x = 2e-6 and 18e-6
vline!(p, [1e-6, 3e-6], color=:red, linestyle=:dash,label="")

# Display the updated plot
display(p)

In [ ]:


g = zeros(Float64, 221)

for c in 1:1:221
    g[c] = abs(real(uh(VectorValue((c-1)*δ/10,-2δ,0.5δ)))[3])
end

# p = plot((0:1:200)*δ/10, g, label="With PML", xlims=(0,200*δ/10), ylims=(-4e-24,4e-24),
#          xlabel="x/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:5e-6:20e-6,yticks=-4e-24:2e-24:4e-24)
p = plot((0:1:220)*δ/10, g, label="With PML", xlims=(0,220*δ/10), 
         xlabel="x/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:2e-6:22e-6,yticks=0:0.4e-24:2e-24)
# Add vertical lines at x = 2e-6 and 18e-6
vline!(p, [2e-6, 20e-6], color=:red, linestyle=:dash,label="")

# Display the updated plot
display(p)


In [ ]:
# Scalar value used for inverse design

result_freq = Vector{Float64}()
result_A = Vector{Float64}()
result_B = Vector{Float64}()

A = BoundaryTriangulation(model, tags="PortA") # Boundary triangulation assumes neumann boundary
dA = CellQuadrature(A, degree)
int1 = sum(integrate(uh, dA ))
int2 = sum(integrate(1,dA))
response_A = int1/int2 /0.001 # 
resZ_A = abs(response_A[3])
append!(result_A, resZ_A)



B = BoundaryTriangulation(model, tags="PortB")
dB = CellQuadrature(B, degree)
int1 = sum(integrate(uh, dB ))
int2 = sum(integrate(1,dB))
response_B = int1/int2 /0.001
resZ_B = abs(response_B[3])
append!(result_B, resZ_B)


println("$freq, $resZ_A, $resZ_B")